In [32]:
import pickle
import pandas as pd

# horse_result_tableを成型
スクレイピングしたrace_idから日付を確認して、日付範囲の成績より未来の成績を入れないようにする

In [43]:
with open(r"C:\Users\yasak\デスクトップ\Keiba_App\mykeibaAI_ver1p0\data\horse_result_table.pkl", "rb") as f:
    horse_result_table = pickle.load(f)
horse_result_table.tail()

,horse_id,race_date,weather,race_no,race_id,num_of_horses,frame_no,horse_no,odds,popularity,...,course_state,distance_length,passing_1st,passing_2nd,passing_3rd,passing_4th,pace_1st,pace_2nd,body_weight,body_diff
178960,2018100141,20200729,0.0,5,202030072905,7,6,6,2.1,1,...,1,1700,7.0,7.0,5.0,1.0,NaN,NaN,444.0,0.0
178961,2018100141,20200701,1.0,9,202030070109,9,8,9,3.3,2,...,1,1700,9.0,9.0,9.0,8.0,NaN,NaN,444.0,-6.0
178962,2018100141,20200618,0.0,5,202030061805,12,6,8,14.4,4,...,1,1700,11.0,11.0,9.0,5.0,NaN,NaN,450.0,0.0
178963,2018100141,20200604,0.0,2,202030060402,11,6,6,4.4,2,...,1,1700,9.0,8.0,10.0,9.0,NaN,NaN,450.0,10.0
178964,2018100141,20200505,0.0,5,202030050505,10,5,5,83.3,10,...,1,1100,10.0,9.0,NaN,NaN,NaN,NaN,440.0,0.0


In [ ]:
# 除外前の確認
horse_result_table["race_id"].astype(str).str.len().value_counts()

race_id
12    178676
4        289
Name: count, dtype: int64

In [ ]:
# 海外レース（race_idが12桁でスクレイピングできなかったレース）を除外
horse_result_table_filtered = horse_result_table[horse_result_table["race_id"].astype(str).str.fullmatch(r"\d{12}")]

In [ ]:
# 除外できているか確認
horse_result_table_filtered["race_id"].astype(str).str.len().value_counts()

race_id
12    178676
Name: count, dtype: int64

In [ ]:
# csvを開いてスクレイピング期間の上限下限を確認
with open(r"C:\Users\yasak\デスクトップ\Keiba_App\mykeibaAI_ver1p0\data\race_id_list_2310_2404.csv") as f:
    race_id_list = f.read().splitlines()

# print(f"race_id_list[0]: {race_id_list[0]}")
# print(f"race_id_list[1]: {race_id_list[1]}")
print(f"race_id_list[-1]: {race_id_list[-1]}")

race_id_list[0]: date,race_id
race_id_list[1]: 20231001,202309040910
race_id_list[-1]: 20240428,202408030411


In [ ]:
# 先頭と末尾の日付を確認
# date_head = race_id_list[1].split(",")[0]
# print(f"date_head: {date_head}")
date_tail = race_id_list[-1].split(",")[0]
print(f"date_tail: {date_tail}")

date_head: 20231001
date_tail: 20240428


In [44]:
# pandasのdatetime型に変換してからフィルタリング
horse_result_table_filtered["race_date"] = pd.to_datetime(horse_result_table_filtered["race_date"])
df = horse_result_table_filtered[(horse_result_table_filtered["race_date"] <= pd.to_datetime(date_tail))]

In [47]:
# 指定の日付範囲で抽出できているか確認
print(horse_result_table_filtered["race_date"].head(30))
print("-----")
print(df["race_date"].head(30))

0    2024-03-23
1    2024-01-14
2    2023-12-23
3    2023-10-28
4    2023-10-01
5    2023-07-08
6    2023-05-13
7    2023-03-25
8    2023-02-12
9    2023-01-09
10   2022-12-17
11   2022-09-19
12   2022-07-16
13   2022-06-19
14   2021-10-17
15   2021-09-20
16   2021-07-29
17   2021-05-22
18   2021-05-01
19   2021-02-13
20   2021-01-24
21   2025-09-27
22   2025-08-23
23   2025-04-19
24   2025-03-30
25   2025-01-26
26   2024-12-28
27   2024-11-24
28   2024-07-06
29   2024-06-16
Name: race_date, dtype: datetime64[ns]
-----
0    2024-03-23
1    2024-01-14
2    2023-12-23
3    2023-10-28
4    2023-10-01
5    2023-07-08
6    2023-05-13
7    2023-03-25
8    2023-02-12
9    2023-01-09
10   2022-12-17
11   2022-09-19
12   2022-07-16
13   2022-06-19
14   2021-10-17
15   2021-09-20
16   2021-07-29
17   2021-05-22
18   2021-05-01
19   2021-02-13
20   2021-01-24
30   2024-04-14
31   2024-03-17
32   2023-12-28
33   2023-11-05
34   2023-10-01
35   2023-09-02
36   2023-05-06
37   2023-04-15
38   2023-0

In [48]:
# 期間内のhorse_result_tableをpickle形式で保存
df.to_pickle(r"C:\Users\yasak\デスクトップ\Keiba_App\mykeibaAI_ver1p0\data\horse_result_table_2404.pkl")

# race_result_tableを成型
正しくスクレイピングできていないデータ（海外レース）を除外

In [49]:
df_race = pd.read_pickle(r"C:\Users\yasak\デスクトップ\Keiba_App\mykeibaAI_ver1p0\data\race_result_table.pkl")
df_race

,date,race_id,is_win,rank,horse_id,weight_carried,jockey_id,popularity,odds,last3f,trainer_id,body_weight,sex,age,time_sec,body_diff
0,20231001,202309040910,1,1,2019105746,58.0,05203,5.0,7.2,35.9,01127,494.0,0,4.0,111.3,-4.0
1,20231001,202309040910,0,2,2019102983,58.0,01093,8.0,22.2,36.3,01069,494.0,0,4.0,111.4,-6.0
2,20231001,202309040910,0,3,2018104746,58.0,01174,4.0,7.0,36.8,00429,494.0,2,5.0,111.5,0.0
3,20231001,202309040910,0,4,2019100108,58.0,01157,11.0,33.9,37.1,01039,494.0,0,4.0,111.6,0.0
4,20231001,202309040910,0,5,2019103518,58.0,01128,10.0,30.1,36.8,00425,532.0,0,4.0,111.6,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28501,20240428,202408030411,0,14,2018104609,58.0,01157,8.0,39.8,39.4,01070,482.0,2,6.0,199.0,-4.0
28502,20240428,202408030411,0,15,2020103650,58.0,05386,2.0,2.8,40.5,01103,464.0,0,4.0,199.8,-8.0
28503,20240428,202408030411,0,16,2016104946,58.0,05212,7.0,34.6,40.3,01071,462.0,0,8.0,200.0,8.0
28504,20240428,202408030411,0,中止,2019100630,58.0,01115,15.0,199.1,NaN,01058,486.0,0,5.0,NaN,2.0


In [50]:
df_race["race_id"].astype(str).str.len().value_counts()

race_id
12    28506
Name: count, dtype: int64

In [52]:
with open(r"C:\Users\yasak\デスクトップ\Keiba_App\mykeibaAI_ver1p0\data\race_id_list_2310_2404.csv", "rb") as f:
    csv = f.read().splitlines()

print(csv)

[b'date,race_id', b'20231001,202309040910', b'20231001,202306040903', b'20231001,202306040912', b'20231001,202309040911', b'20231001,202306040909', b'20231001,202306040906', b'20231001,202309040901', b'20231001,202306040910', b'20231001,202309040906', b'20231001,202306040901', b'20231001,202306040904', b'20231001,202306040902', b'20231001,202309040912', b'20231001,202309040908', b'20231001,202309040905', b'20231001,202306040907', b'20231001,202309040902', b'20231001,202309040909', b'20231001,202309040907', b'20231001,202309040904', b'20231001,202306040908', b'20231001,202309040903', b'20231001,202306040911', b'20231001,202306040905', b'20231007,202305040109', b'20231007,202308020107', b'20231007,202305040103', b'20231007,202308020106', b'20231007,202308020109', b'20231007,202308020110', b'20231007,202308020111', b'20231007,202308020112', b'20231007,202305040105', b'20231007,202305040106', b'20231007,202305040107', b'20231007,202305040108', b'20231007,202308020105', b'20231007,202308020

race_result_tableを確認したところ、race_idに不完全なものは存在しない
→csvを確認しても不完全なrace_idは存在しないため、race_idをカレンダーから抽出した時にすでに省かれている？